In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=3b80fadc4392725daa4201d3dcb398a5a23e5d598077894baf8a2d3d41dba67c
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


In [2]:
# Import Apache Spark SQL
from pyspark.sql import SparkSession

# Create Spark Session/Context
# We are using local machine with all the CPU cores [*]
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Hello Pyspark") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [3]:
# Check spark session
print(spark)

In [4]:
spark.sparkContext.setLogLevel("ERROR")

In [5]:
from pyspark.sql.types import *
df_schema = StructType([StructField("link", StringType(), True)\
                       ,StructField("headline", StringType(), True)\
                       ,StructField("category", StringType(), True)\
                       ,StructField("short_description", StringType(), True)\
                       ,StructField("authors", StringType(), True)\
                       ,StructField("date", StringType(), True)])

In [6]:
df = spark.readStream.schema(df_schema).json("/content/input-05111940000202/")

In [7]:
groupDF = df.select("category").groupBy("category").count()

groupDF.printSchema()

root
 |-- category: string (nullable = true)
 |-- count: long (nullable = false)



In [10]:
result = groupDF.writeStream\
  .format("json")\
  .option("checkpointLocation", "checkpoint")\
  .option("path", "output-05111940000202")\
  .outputMode("append")\
  .start()


result.awaitTermination()

AnalysisException: ignored